In [25]:
import pandas as pd
import numpy as np
from fpdf import FPDF
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import date

df_xg = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/xg.csv')

df_possession_stats = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/terr_poss.csv')

df_xg_agg = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/xg_agg.csv')

penalty_area_entry_counts = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/penalty_area_entry_counts.csv')

df_matchstats = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/matchstats.csv')

df_ppda = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/ppda.csv')
    
events = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/events.csv')

In [29]:
def Process_data_spillere(events,df_xg,df_matchstats):
    xg = events[['player.name','label','shot.xg']]
    xg['shot.xg'] = xg['shot.xg'].astype(float)
    xg = xg.groupby(['player.name','label']).sum().reset_index()
    df_scouting = xg.merge(df_matchstats, on=['player.name', 'label'], how='inner')
    def calculate_score(df, column, score_column):
        df_unique = df.drop_duplicates(column).copy()
        df_unique.loc[:, score_column] = pd.qcut(df_unique[column], q=10, labels=False, duplicates='raise') + 1
        return df.merge(df_unique[[column, score_column]], on=column, how='left')
    
    minutter_kamp = 45
    minutter_total = 300
    
    df_matchstats = df_matchstats[['player.name','team.name','label','position_codes','total_minutesOnField','average_successfulPassesToFinalThird','percent_aerialDuelsWon','percent_newSuccessfulDribbles','average_throughPasses','percent_duelsWon','percent_successfulPassesToFinalThird','average_xgAssist','average_crosses','average_progressivePasses','average_progressiveRun','average_accelerations','average_passesToFinalThird','percent_successfulProgressivePasses','percent_successfulPasses','average_ballRecoveries','average_interceptions','average_defensiveDuels','average_successfulDefensiveAction','average_forwardPasses','average_successfulForwardPasses','average_touchInBox','average_xgShot','average_keyPasses','average_successfulAttackingActions','average_shotAssists']]
    df_scouting = df_xg.merge(df_matchstats,how='right')
    df_scouting['penAreaEntries_per90&crosses%shotassists'] = ((df_scouting['average_passesToFinalThird'].astype(float)+df_scouting['average_crosses'].astype(float) + df_scouting['average_xgAssist'].astype(float))/ df_scouting['total_minutesOnField'].astype(float)) * 90

    df_scouting.fillna(0, inplace=True)
    df_scouting = df_scouting.drop_duplicates(subset=['player.name', 'team.name', 'position_codes','label'])

    def ball_playing_central_defender():
        df_spillende_stopper = df_scouting[df_scouting['player_codes'].str.contains('cb')]
        df_spillende_stopper['total_minutesOnField'] = df_spillende_stopper['total_minutesOnField'].astype(int)
        df_spillende_stopper = df_spillende_stopper[df_spillende_stopper['total_minutesOnField'].astype(int) >= minutter_kamp]
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'percent_duelsWon', 'percent_duelsWon score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'average_interceptions', 'average_interceptions score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'average_ballRecoveries', 'average_ballRecoveries score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'average_successfulAttackingActions', 'average_successfulAttackingActions score')
        df_spillende_stopper = calculate_score(df_spillende_stopper, 'percent_aerialDuelsWon', 'percent_aerialDuelsWon score')

        df_spillende_stopper['Passing'] = df_spillende_stopper[['percent_successfulPasses score', 'percent_successfulPasses score']].mean(axis=1)
        df_spillende_stopper['Forward passing'] = df_spillende_stopper[['percent_successfulPassesToFinalThird score']].mean(axis=1)
        df_spillende_stopper['Defending'] = df_spillende_stopper[['percent_duelsWon score', 'average_interceptions score', 'average_interceptions score', 'average_ballRecoveries score']].mean(axis=1)
        df_spillende_stopper['Possession value added'] = df_spillende_stopper['average_successfulAttackingActions score']
        
        df_spillende_stopper['Total score'] = df_spillende_stopper[['Passing','Passing','Forward passing','Forward passing','Forward passing','Defending','Defending','Possession value added','Possession value added','Possession value added']].mean(axis=1)
        df_spillende_stopper = df_spillende_stopper[['player.name','team.name','position_codes','label','total_minutesOnField','Passing','Forward passing','Defending','Possession value added score','Total score']] 
        df_spillende_stoppertotal = df_spillende_stopper[['player.name','team.name','position_codes','total_minutesOnField','Passing','Forward passing','Defending','Possession value added score','Total score']]
        df_spillende_stoppertotal = df_spillende_stoppertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_spillende_stopper.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_spillende_stoppertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_spillende_stopper = df_spillende_stopper.sort_values('Total score',ascending = False)
        df_spillende_stoppertotal = df_spillende_stoppertotal[['player.name','team.name','position_codes','total_minutesOnField total','Passing','Forward passing','Defending','Possession value added score','Total score']]
        df_spillende_stoppertotal = df_spillende_stoppertotal[df_spillende_stoppertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_spillende_stoppertotal = df_spillende_stoppertotal.sort_values('Total score',ascending = False)
        return df_spillende_stopper
  
    def defending_central_defender():
        df_forsvarende_stopper = df_scouting[df_scouting['player_codes'].str.contains('cb')]
        df_forsvarende_stopper['total_minutesOnField'] = df_forsvarende_stopper['total_minutesOnField'].astype(int)
        df_forsvarende_stopper = df_forsvarende_stopper[df_forsvarende_stopper['total_minutesOnField'].astype(int) >= minutter_kamp]
        
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper, 'percent_duelsWon', 'percent_duelsWon score')
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper, 'average_interceptions', 'average_interceptions score')
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper, 'average_ballRecoveries', 'ballRecovery score')
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper,'percent_aerialDuelsWon', 'percent_aerialDuelsWon score')
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper, 'average_successfulAttackingActions', 'average_successfulAttackingActions score')
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_forsvarende_stopper = calculate_score(df_forsvarende_stopper, 'percent_successfulPasses', 'percent_successfulPasses score')


        df_forsvarende_stopper['Defending'] = df_forsvarende_stopper[['percent_duelsWon score','percent_aerialDuelsWon score', 'average_interceptions score', 'average_interceptions score', 'ballRecovery score']].mean(axis=1)
        df_forsvarende_stopper['Duels'] = df_forsvarende_stopper[['percent_duelsWon score','percent_duelsWon score','percent_aerialDuelsWon score']].mean(axis=1)
        df_forsvarende_stopper['Intercepting'] = df_forsvarende_stopper[['average_interceptions score','average_interceptions score','ballRecovery score']].mean(axis=1)
        df_forsvarende_stopper['Passing'] = df_forsvarende_stopper[['percent_successfulPasses score', 'percent_successfulPasses score','average_successfulAttackingActions score','average_successfulAttackingActions score']].mean(axis=1)
        df_forsvarende_stopper['Total score'] = df_forsvarende_stopper[['Defending','Defending','Defending','Defending','Duels','Duels','Duels','Intercepting','Intercepting','Intercepting','Passing','Passing']].mean(axis=1)

        df_forsvarende_stopper = df_forsvarende_stopper[['player.name','team.name','position_codes','label','total_minutesOnField','Defending','Duels','Intercepting','Passing','Total score']]
        df_forsvarende_stoppertotal = df_forsvarende_stopper[['player.name','team.name','position_codes','total_minutesOnField','Defending','Duels','Intercepting','Passing','Total score']]
        df_forsvarende_stoppertotal = df_forsvarende_stoppertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_forsvarende_stopper.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_forsvarende_stoppertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_forsvarende_stopper = df_forsvarende_stopper.sort_values('Total score',ascending = False)
        df_forsvarende_stoppertotal = df_forsvarende_stoppertotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending','Duels','Intercepting','Passing','Total score']]
        df_forsvarende_stoppertotal = df_forsvarende_stoppertotal[df_forsvarende_stoppertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_forsvarende_stoppertotal = df_forsvarende_stoppertotal.sort_values('Total score',ascending = False)
        return df_forsvarende_stopper

    def balanced_central_defender():
        df_balanced_central_defender = df_scouting[df_scouting['position_codes'].str.contains(['cb'])]
        df_balanced_central_defender['total_minutesOnField'] = df_balanced_central_defender['total_minutesOnField'].astype(int)
        df_balanced_central_defender = df_balanced_central_defender[df_balanced_central_defender['total_minutesOnField'].astype(int) >= minutter_kamp]
        
        df_balanced_central_defender = calculate_score(df_balanced_central_defender, 'percent_duelsWon', 'percent_duelsWon score')
        df_balanced_central_defender = calculate_score(df_balanced_central_defender, 'average_interceptions', 'average_interceptions score')
        df_balanced_central_defender = calculate_score(df_balanced_central_defender, 'average_ballRecoveries', 'ballRecovery score')
        df_balanced_central_defender = calculate_score(df_balanced_central_defender,'percent_aerialDuelsWon', 'percent_aerialDuelsWon score')
        df_balanced_central_defender = calculate_score(df_balanced_central_defender,'average_successfulAttackingActions', 'average_successfulAttackingActions score')
        df_balanced_central_defender = calculate_score(df_balanced_central_defender, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_balanced_central_defender = calculate_score(df_balanced_central_defender, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')

        df_balanced_central_defender['Defending'] = df_balanced_central_defender[['percent_duelsWon score','percent_aerialDuelsWon score', 'average_interceptions score', 'average_interceptions score', 'ballRecovery score']].mean(axis=1)
        df_balanced_central_defender['Possession value added'] = df_balanced_central_defender['average_successfulAttackingActions score']
        df_balanced_central_defender['Passing'] = df_balanced_central_defender[['percent_successfulPasses score', 'percent_successfulPasses score','percent_successfulPassesToFinalThird score']].mean(axis=1)
        
        df_balanced_central_defender['Total score'] = df_balanced_central_defender[['Defending','Possession value added','Passing']].mean(axis=1)

        df_balanced_central_defender = df_balanced_central_defender[['player.name','team.name','position_codes','label','total_minutesOnField','Defending','Possession value added','Passing','Total score']]
        
        df_balanced_central_defendertotal = df_balanced_central_defender[['player.name','team.name','position_codes','total_minutesOnField','Defending','Possession value added','Passing','Total score']]
        df_balanced_central_defendertotal = df_balanced_central_defendertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_balanced_central_defender.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_balanced_central_defendertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_balanced_central_defender = df_balanced_central_defender.sort_values('Total score',ascending = False)
        df_balanced_central_defendertotal = df_balanced_central_defendertotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending','Possession value added','Passing','Total score']]
        df_balanced_central_defendertotal = df_balanced_central_defendertotal[df_balanced_central_defendertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_balanced_central_defendertotal = df_balanced_central_defendertotal.sort_values('Total score',ascending = False)
        return df_balanced_central_defender
    
    def fullbacks():
        df_backs = df_scouting[(df_scouting['position_codes'].str.contains('rb') |df_scouting['position_codes'].str.contains('lb') |df_scouting['position_codes'].str.contains('lwb') |df_scouting['position_codes'].str.contains('rwb'))]        
        df_backs['total_minutesOnField'] = df_backs['total_minutesOnField'].astype(int)
        df_backs = df_backs[df_backs['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_backs = calculate_score(df_backs,'average_successfulAttackingActions', 'Possession value added score')
        df_backs = calculate_score(df_backs, 'percent_duelsWon', 'percent_duelsWon score')
        df_backs = calculate_score(df_backs, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_backs = calculate_score(df_backs, 'penAreaEntries_per90&crosses%shotassists', 'Penalty area entries & crosses & shot assists score')
        df_backs = calculate_score(df_backs, 'average_shotAssists', 'average_shotAssists score')
        df_backs = calculate_score(df_backs, 'average_interceptions', 'interception_per90 score')
        df_backs = calculate_score(df_backs, 'average_interceptions', 'average_interceptions score')
        df_backs = calculate_score(df_backs, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_backs = calculate_score(df_backs, 'average_crosses', 'average_crosses_per90 score')
        df_backs['Defending'] = (df_backs['percent_duelsWon score'] + df_backs['percent_duelsWon score'] + df_backs['average_interceptions score'])/3
        df_backs['Passing'] = (df_backs['percent_successfulPassesToFinalThird score'] + df_backs['percent_successfulPassesToFinalThird score'] + df_backs['percent_successfulPasses score'] + df_backs['Possession value added score'])/4
        df_backs['Chance creation'] = (df_backs['Penalty area entries & crosses & shot assists score'] + df_backs['average_crosses_per90 score'] + df_backs['average_crosses_per90 score'] + df_backs['percent_successfulPassesToFinalThird score']+ df_backs['percent_successfulPassesToFinalThird score'] + df_backs['percent_successfulPassesToFinalThird score']+ df_backs['percent_successfulPassesToFinalThird score'] + df_backs['Possession value added score'] + df_backs['Possession value added score'])/9
        df_backs['Possession value added'] = df_backs['Possession value added score']
        
        df_backs = calculate_score(df_backs, 'Defending', 'Defending_')
        df_backs = calculate_score(df_backs, 'Passing', 'Passing_')
        df_backs = calculate_score(df_backs, 'Chance creation','Chance_creation')
        df_backs = calculate_score(df_backs, 'Possession value added', 'Possession_value_added')
        
        df_backs['Total score'] = (df_backs['Defending_'] + df_backs['Defending_'] + df_backs['Defending_'] + df_backs['Defending_'] + df_backs['Passing_']+ df_backs['Passing_'] + df_backs['Chance_creation'] + df_backs['Chance_creation'] + df_backs['Chance_creation'] + df_backs['Possession_value_added'] + df_backs['Possession_value_added'] + df_backs['Possession_value_added'] + df_backs['Possession_value_added']) / 13
        df_backs = df_backs[['player.name','team.name','position_codes','label','total_minutesOnField','Defending_','Passing_','Chance_creation','Possession_value_added','Total score']]
        df_backs = df_backs.dropna()
        df_backstotal = df_backs[['player.name','team.name','position_codes','total_minutesOnField','Defending_','Passing_','Chance_creation','Possession_value_added','Total score']]
        df_backstotal = df_backstotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_backs.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_backstotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_backs = df_backs.sort_values('Total score',ascending = False)
        df_backstotal = df_backstotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending_','Passing_','Chance_creation','Possession_value_added','Total score']]
        df_backstotal = df_backstotal[df_backstotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_backstotal = df_backstotal.sort_values('Total score',ascending = False)
        return df_backs
    
    def number6():
        df_sekser = df_scouting[(df_scouting['position_codes'].str.contains('dmf'))]
        df_sekser['total_minutesOnField'] = df_sekser['total_minutesOnField'].astype(int)
        df_sekser = df_sekser[df_sekser['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_sekser = calculate_score(df_sekser,'average_successfulAttackingActions', 'Possession value added score')
        df_sekser = calculate_score(df_sekser, 'percent_duelsWon', 'percent_duelsWon score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_sekser = calculate_score(df_sekser, 'average_interceptions', 'average_interceptions score')
        df_sekser = calculate_score(df_sekser, 'average_ballRecoveries', 'possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90 score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_sekser = calculate_score(df_sekser, 'average_ballRecoveries', 'ballRecovery score')

        
        df_sekser['Defending'] = df_sekser[['percent_duelsWon score','average_interceptions score','average_interceptions score','ballRecovery score']].mean(axis=1)
        df_sekser['Passing'] = df_sekser[['percent_successfulPasses score','percent_successfulPasses score']].mean(axis=1)
        df_sekser['Progressive ball movement'] = df_sekser[['Possession value added score','Possession value added score','percent_successfulPassesToFinalThird score']].mean(axis=1)
        df_sekser['Possession value added'] = df_sekser['Possession value added score']
        
        df_sekser = calculate_score(df_sekser, 'Defending', 'Defending_')
        df_sekser = calculate_score(df_sekser, 'Passing', 'Passing_')
        df_sekser = calculate_score(df_sekser, 'Progressive ball movement','Progressive_ball_movement')
        df_sekser = calculate_score(df_sekser, 'Possession value added', 'Possession_value_added')
        
        df_sekser['Total score'] = df_sekser[['Defending_','Passing_','Progressive_ball_movement','Possession_value_added']].mean(axis=1)
        df_sekser = df_sekser[['player.name','team.name','position_codes','label','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]
        df_sekser = df_sekser.dropna()
        df_seksertotal = df_sekser[['player.name','team.name','position_codes','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]

        df_seksertotal = df_seksertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_sekser.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_seksertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_sekser = df_sekser.sort_values('Total score',ascending = False)
        df_seksertotal = df_seksertotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]
        df_seksertotal= df_seksertotal[df_seksertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_seksertotal = df_seksertotal.sort_values('Total score',ascending = False)
        return df_sekser

    def number6_destroyer():
        df_sekser = df_scouting[(df_scouting['position_codes'].str.conitans('dmf'))]
        df_sekser['total_minutesOnField'] = df_sekser['total_minutesOnField'].astype(int)
        df_sekser = df_sekser[df_sekser['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_sekser = calculate_score(df_sekser,'average_successfulAttackingActions', 'Possession value added score')
        df_sekser = calculate_score(df_sekser, 'percent_duelsWon', 'percent_duelsWon score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_sekser = calculate_score(df_sekser, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_sekser = calculate_score(df_sekser, 'average_interceptions', 'average_interceptions score')
        df_sekser = calculate_score(df_sekser, 'possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90', 'possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90 score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_sekser = calculate_score(df_sekser, 'average_ballRecoveries', 'ballRecovery score')

        
        df_sekser['Defending'] = df_sekser[['percent_duelsWon score','average_interceptions score','average_interceptions score','ballRecovery score']].mean(axis=1)
        df_sekser['Passing'] = df_sekser[['percent_successfulPasses score','percent_successfulPasses score']].mean(axis=1)
        df_sekser['Progressive ball movement'] = df_sekser[['Possession value added score','Possession value added score','percent_successfulPassesToFinalThird score']].mean(axis=1)
        df_sekser['Possession value added'] = df_sekser['Possession value added score']
        
        df_sekser = calculate_score(df_sekser, 'Defending', 'Defending_')
        df_sekser = calculate_score(df_sekser, 'Passing', 'Passing_')
        df_sekser = calculate_score(df_sekser, 'Progressive ball movement','Progressive_ball_movement')
        df_sekser = calculate_score(df_sekser, 'Possession value added', 'Possession_value_added')
        
        df_sekser['Total score'] = df_sekser[['Defending_','Defending_','Defending_','Passing_','Passing_','Progressive_ball_movement','Possession_value_added']].mean(axis=1)
        df_sekser = df_sekser[['player.name','team.name','position_codes','label','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]
        df_sekser = df_sekser.dropna()

        df_seksertotal = df_sekser[['player.name','team.name','position_codes','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]

        df_seksertotal = df_seksertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_sekser.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_seksertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_sekser_destroyer = df_sekser.sort_values('Total score',ascending = False)
        df_seksertotal = df_seksertotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]
        df_seksertotal= df_seksertotal[df_seksertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_seksertotal = df_seksertotal.sort_values('Total score',ascending = False)
        return df_sekser_destroyer
    
    def number6_double_6_forward():
        df_sekser = df_scouting[(df_scouting['position_codes'].str.conitans('dmf'))]
        df_sekser['total_minutesOnField'] = df_sekser['total_minutesOnField'].astype(int)
        df_sekser = df_sekser[df_sekser['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_sekser = calculate_score(df_sekser,'average_successfulAttackingActions', 'Possession value added score')
        df_sekser = calculate_score(df_sekser, 'percent_duelsWon', 'percent_duelsWon score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_sekser = calculate_score(df_sekser, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_sekser = calculate_score(df_sekser, 'average_interceptions', 'average_interceptions score')
        df_sekser = calculate_score(df_sekser, 'possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90', 'possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90 score')
        df_sekser = calculate_score(df_sekser, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_sekser = calculate_score(df_sekser, 'average_ballRecoveries', 'ballRecovery score')

        
        df_sekser['Defending'] = df_sekser[['percent_duelsWon score','average_interceptions score','average_interceptions score','ballRecovery score']].mean(axis=1)
        df_sekser['Passing'] = df_sekser[['percent_successfulPasses score','percent_successfulPasses score']].mean(axis=1)
        df_sekser['Progressive ball movement'] = df_sekser[['Possession value added score','Possession value added score','percent_successfulPassesToFinalThird score']].mean(axis=1)
        df_sekser['Possession value added'] = df_sekser['Possession value added score']
        
        df_sekser = calculate_score(df_sekser, 'Defending', 'Defending_')
        df_sekser = calculate_score(df_sekser, 'Passing', 'Passing_')
        df_sekser = calculate_score(df_sekser, 'Progressive ball movement','Progressive_ball_movement')
        df_sekser = calculate_score(df_sekser, 'Possession value added', 'Possession_value_added')
        
        df_sekser['Total score'] = df_sekser[['Defending_','Defending_','Passing_','Passing_','Progressive_ball_movement','Progressive_ball_movement','Possession_value_added','Possession_value_added']].mean(axis=1)
        df_sekser = df_sekser[['player.name','team.name','position_codes','label','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]
        df_sekser = df_sekser.dropna()
        df_seksertotal = df_sekser[['player.name','team.name','position_codes','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]

        df_seksertotal = df_seksertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_sekser.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_seksertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_sekser_double_6_forward = df_sekser.sort_values('Total score',ascending = False)
        df_seksertotal = df_seksertotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending_','Passing_','Progressive_ball_movement','Possession_value_added','Total score']]
        df_seksertotal= df_seksertotal[df_seksertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_seksertotal = df_seksertotal.sort_values('Total score',ascending = False)
        return df_sekser_double_6_forward
    
    def number8():
        df_otter = df_scouting[(df_scouting['position_codes'].str.contains('cmf'))]
        df_otter['total_minutesOnField'] = df_otter['total_minutesOnField'].astype(int)
        df_otter = df_otter[df_otter['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_otter = calculate_score(df_otter,'average_successfulAttackingActions','Possession value total score')
        df_otter = calculate_score(df_otter,'average_successfulAttackingActions', 'Possession value score')
        df_otter = calculate_score(df_otter,'average_successfulAttackingActions', 'Possession value added score')
        df_otter = calculate_score(df_otter, 'percent_duelsWon', 'percent_duelsWon score')
        df_otter = calculate_score(df_otter, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_otter = calculate_score(df_otter, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_otter = calculate_score(df_otter, 'average_interceptions', 'average_interceptions score')
        df_otter = calculate_score(df_otter, 'average_ballRecoveries', 'possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90 score')
        df_otter = calculate_score(df_otter, 'average_passesToFinalThird', 'average_passesToFinalThird score')
        df_otter = calculate_score(df_otter, 'average_shotAssists','average_shotAssists score')
        df_otter = calculate_score(df_otter, 'average_touchInBox','average_touchInBox score')
        df_otter = calculate_score(df_otter, 'average_progressivePasses', 'average_progressivePasses score')


        df_otter['Defending'] = df_otter[['percent_duelsWon score','possWonDef3rd_possWonMid3rd_possWonAtt3rd_per90 score']].mean(axis=1)
        df_otter['Passing'] = df_otter[['percent_successfulPassesToFinalThird score','percent_successfulPasses score']].mean(axis=1)
        df_otter['Progressive ball movement'] = df_otter[['average_shotAssists score','average_progressivePasses score','average_touchInBox score','percent_successfulPassesToFinalThird score','Possession value total score']].mean(axis=1)
        df_otter['Possession value'] = df_otter[['Possession value added score','Possession value total score']].mean(axis=1)
        
        df_otter = calculate_score(df_otter, 'Defending', 'Defending_')
        df_otter = calculate_score(df_otter, 'Passing', 'Passing_')
        df_otter = calculate_score(df_otter, 'Progressive ball movement','Progressive_ball_movement')
        df_otter = calculate_score(df_otter, 'Possession value', 'Possession_value')
        
        df_otter['Total score'] = df_otter[['Defending_','Passing_','Passing_','Progressive_ball_movement','Progressive_ball_movement','Possession_value','Possession_value','Possession_value']].mean(axis=1)
        df_otter = df_otter[['player.name','team.name','position_codes','label','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value','Total score']]
        df_otter = df_otter.dropna()

        df_ottertotal = df_otter[['player.name','team.name','position_codes','total_minutesOnField','Defending_','Passing_','Progressive_ball_movement','Possession_value','Total score']]

        df_ottertotal = df_ottertotal.groupby(['player.name','team.name','position_codes']).mean().reset_index()
        minutter = df_otter.groupby(['player.name', 'team.name','position_codes'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_ottertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_otter = df_otter.sort_values('Total score',ascending = False)
        df_ottertotal = df_ottertotal[['player.name','team.name','position_codes','total_minutesOnField total','Defending_','Passing_','Progressive_ball_movement','Possession_value','Total score']]
        df_ottertotal= df_ottertotal[df_ottertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_ottertotal = df_ottertotal.sort_values('Total score',ascending = False)
        return df_otter
        
    def number10():
        df_10 = df_scouting[(df_scouting['position_codes'].str.contains('amf'))]
        df_10['total_minutesOnField'] = df_10['total_minutesOnField'].astype(int)
        df_10 = df_10[df_10['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_10 = calculate_score(df_10,'average_successfulAttackingActions','Possession value total score')
        df_10 = calculate_score(df_10,'average_successfulAttackingActions', 'Possession value score')
        df_10 = calculate_score(df_10,'average_successfulAttackingActions', 'Possession value added score')
        df_10 = calculate_score(df_10, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_10 = calculate_score(df_10, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_10 = calculate_score(df_10, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_10 = calculate_score(df_10, 'average_progressivePasses', 'average_progressivePasses score')
        df_10 = calculate_score(df_10, 'average_shotAssists','average_shotAssists score')
        df_10 = calculate_score(df_10, 'average_touchInBox','average_touchInBox score')
        df_10 = calculate_score(df_10, 'percent_newSuccessfulDribbles','percent_newSuccessfulDribbles score')
        df_10 = calculate_score(df_10, 'average_throughPasses','average_throughPasses score')
        df_10 = calculate_score(df_10, 'average_keyPasses','average_keyPasses score')
        df_10 = calculate_score(df_10, 'shot.xg','shot.xg score')


        df_10['Passing'] = df_10[['percent_successfulPassesToFinalThird score','percent_successfulPasses score']].mean(axis=1)
        df_10['Chance creation'] = df_10[['average_shotAssists score','average_touchInBox score','percent_successfulPassesToFinalThird score','percent_successfulPassesToFinalThird score','Possession value total score','Possession value score','average_progressivePasses score','percent_newSuccessfulDribbles score','average_touchInBox score','average_throughPasses score','average_keyPasses score']].mean(axis=1)
        df_10['Goalscoring'] = df_10[['average_touchInBox score','shot.xg score','shot.xg score','shot.xg score']].mean(axis=1)
        df_10['Possession value'] = df_10[['Possession value total score','Possession value total score','Possession value added score','Possession value score','Possession value score','Possession value score']].mean(axis=1)
                
        df_10 = calculate_score(df_10, 'Passing', 'Passing_')
        df_10 = calculate_score(df_10, 'Chance creation','Chance_creation')
        df_10 = calculate_score(df_10, 'Goalscoring','Goalscoring_')        
        df_10 = calculate_score(df_10, 'Possession value', 'Possession_value')
        
        df_10['Total score'] = df_10[['Passing_','Chance_creation','Chance_creation','Chance_creation','Chance_creation','Goalscoring_','Goalscoring_','Goalscoring_','Possession_value','Possession_value','Possession_value']].mean(axis=1)
        df_10 = df_10[['player.name','team.name','label','total_minutesOnField','Passing_','Chance_creation','Goalscoring_','Possession_value','Total score']]
        df_10 = df_10.dropna()
        df_10total = df_10[['player.name','team.name','total_minutesOnField','Passing_','Chance_creation','Goalscoring_','Possession_value','Total score']]

        df_10total = df_10total.groupby(['player.name','team.name']).mean().reset_index()
        minutter = df_10.groupby(['player.name', 'team.name'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_10total['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_10 = df_10.sort_values('Total score',ascending = False)
        df_10total = df_10total[['player.name','team.name','total_minutesOnField total','Passing_','Chance_creation','Goalscoring_','Possession_value','Total score']]
        df_10total= df_10total[df_10total['total_minutesOnField total'].astype(int) >= minutter_total]
        df_10total = df_10total.sort_values('Total score',ascending = False)
        return df_10
    
    def winger():
        df_10 = df_scouting[(df_scouting['position_codes'].str.contains('lw')) | (df_scouting['position_codes'].str.contains('rw'))] 
        df_10['total_minutesOnField'] = df_10['total_minutesOnField'].astype(int)
        df_10 = df_10[df_10['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_10 = calculate_score(df_10,'average_successfulAttackingActions','Possession value total score')
        df_10 = calculate_score(df_10,'average_successfulAttackingActions', 'Possession value score')
        df_10 = calculate_score(df_10,'average_successfulAttackingActions', 'Possession value added score')
        df_10 = calculate_score(df_10,'average_progressiveRun', 'progressiveRun score')
        df_10 = calculate_score(df_10, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_10 = calculate_score(df_10, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_10 = calculate_score(df_10, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_10 = calculate_score(df_10, 'average_progressivePasses', 'average_progressivePasses score')
        df_10 = calculate_score(df_10, 'average_shotAssists','average_shotAssists score')
        df_10 = calculate_score(df_10, 'average_touchInBox','average_touchInBox score')
        df_10 = calculate_score(df_10, 'percent_newSuccessfulDribbles','percent_newSuccessfulDribbles score')
        df_10 = calculate_score(df_10, 'average_throughPasses','average_throughPasses score')
        df_10 = calculate_score(df_10, 'average_keyPasses','average_keyPasses score')
        df_10 = calculate_score(df_10, 'shot.xg','shot.xg score')


        df_10['Passing'] = df_10[['percent_successfulPassesToFinalThird score','percent_successfulPasses score']].mean(axis=1)
        df_10['Chance creation'] = df_10[['progressiveRun score','average_shotAssists score','average_touchInBox score','percent_successfulPassesToFinalThird score','percent_successfulPassesToFinalThird score','Possession value total score','Possession value score','percent_newSuccessfulDribbles score','percent_newSuccessfulDribbles score','percent_newSuccessfulDribbles score','average_touchInBox score','average_throughPasses score','average_keyPasses score','average_keyPasses score','average_keyPasses score']].mean(axis=1)
        df_10['Goalscoring'] = df_10[['average_touchInBox','shot.xg score','shot.xg score','shot.xg score']].mean(axis=1)
        df_10['Possession value'] = df_10[['Possession value total score','Possession value total score','Possession value added score','Possession value score','Possession value score','Possession value score']].mean(axis=1)
                
        df_10 = calculate_score(df_10, 'Passing', 'Passing_')
        df_10 = calculate_score(df_10, 'Chance creation','Chance_creation')
        df_10 = calculate_score(df_10, 'Goalscoring','Goalscoring_')        
        df_10 = calculate_score(df_10, 'Possession value', 'Possession_value')
        
        df_10['Total score'] = df_10[['Passing_','Chance_creation','Chance_creation','Chance_creation','Chance_creation','Goalscoring_','Goalscoring_','Goalscoring_','Goalscoring_','Possession_value','Possession_value','Possession_value','Possession_value']].mean(axis=1)
        df_10 = df_10[['player.name','team.name','label','total_minutesOnField','Passing_','Chance_creation','Goalscoring_','Possession_value','Total score']]
        df_10 = df_10.dropna()
        df_10total = df_10[['player.name','team.name','total_minutesOnField','Passing_','Chance_creation','Goalscoring_','Possession_value','Total score']]

        df_10total = df_10total.groupby(['player.name','team.name']).mean().reset_index()
        minutter = df_10.groupby(['player.name', 'team.name'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_10total['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_kant = df_10.sort_values('Total score',ascending = False)
        df_10total = df_10total[['player.name','team.name','total_minutesOnField total','Passing_','Chance_creation','Goalscoring_','Possession_value','Total score']]
        df_10total= df_10total[df_10total['total_minutesOnField total'].astype(int) >= minutter_total]
        df_10total = df_10total.sort_values('Total score',ascending = False)
        return df_kant
    
    def Classic_striker():
        df_striker = df_scouting[(df_scouting['position_codes'].str.contains('cf'))]
        df_striker['total_minutesOnField'] = df_striker['total_minutesOnField'].astype(int)
        df_striker = df_striker[df_striker['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_striker = calculate_score(df_striker,'average_successfulAttackingActions','Possession value total score')
        df_striker = calculate_score(df_striker,'average_successfulAttackingActions', 'Possession value score')
        df_striker = calculate_score(df_striker,'average_successfulAttackingActions', 'Possession value added score')
        df_striker = calculate_score(df_striker, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_striker = calculate_score(df_striker, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'average_progressivePasses', 'average_progressivePasses score')
        df_striker = calculate_score(df_striker, 'average_shotAssists','average_shotAssists score')
        df_striker = calculate_score(df_striker, 'average_touchInBox','average_touchInBox score')
        df_striker = calculate_score(df_striker, 'percent_newSuccessfulDribbles','newSuccessfulDribbles score')
        df_striker = calculate_score(df_striker, 'average_keyPasses','average_keyPasses score')
        df_striker = calculate_score(df_striker, 'shot.xg','shot.xg score')


        df_striker['Linkup_play'] = df_striker[['percent_successfulPassesToFinalThird score','percent_successfulPasses score','Possession value score','average_touchInBox score','average_successfulPassesToFinalThird score']].mean(axis=1)
        df_striker['Chance_creation'] = df_striker[['average_touchInBox score','Possession value total score','average_touchInBox score','average_successfulPassesToFinalThird score']].mean(axis=1)
        df_striker['Goalscoring_'] = df_striker[['average_touchInBox','shot.xg score','shot.xg score','shot.xg score','shot.xg score','shot.xg score']].mean(axis=1)
        df_striker['Possession_value'] = df_striker[['Possession value total score','Possession value score','Possession value score','Possession value score']].mean(axis=1)

        df_striker = calculate_score(df_striker, 'Linkup_play', 'Linkup play')
        df_striker = calculate_score(df_striker, 'Chance_creation','Chance creation')
        df_striker = calculate_score(df_striker, 'Goalscoring_','Goalscoring')        
        df_striker = calculate_score(df_striker, 'Possession_value', 'Possession value')

        
        df_striker['Total score'] = df_striker[['Linkup play','Chance creation','Goalscoring','Possession value']].mean(axis=1)
        df_striker = df_striker[['player.name','team.name','label','total_minutesOnField','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]
        df_striker = df_striker.dropna()

        df_strikertotal = df_striker[['player.name','team.name','total_minutesOnField','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]

        df_strikertotal = df_strikertotal.groupby(['player.name','team.name']).mean().reset_index()
        minutter = df_striker.groupby(['player.name', 'team.name'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_strikertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_classic_striker = df_striker.sort_values('Total score',ascending = False)
        df_strikertotal = df_strikertotal[['player.name','team.name','total_minutesOnField total','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]
        df_strikertotal= df_strikertotal[df_strikertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_strikertotal = df_strikertotal.sort_values('Total score',ascending = False)
        return df_classic_striker
    
    def Targetman():
        df_striker = df_scouting[(df_scouting['position_codes'] == 'Striker') & (df_scouting['position_codesSide'].str.contains('Centre'))]
        df_striker['total_minutesOnField'] = df_striker['total_minutesOnField'].astype(int)
        df_striker = df_striker[df_striker['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_striker = calculate_score(df_striker,'average_successfulAttackingActions','Possession value total score')
        df_striker = calculate_score(df_striker,'average_successfulAttackingActions', 'Possession value score')
        df_striker = calculate_score(df_striker,'average_successfulAttackingActions', 'Possession value added score')
        df_striker = calculate_score(df_striker, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_striker = calculate_score(df_striker, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'average_progressivePasses', 'average_progressivePasses score')
        df_striker = calculate_score(df_striker, 'average_shotAssists','average_shotAssists score')
        df_striker = calculate_score(df_striker, 'average_touchInBox','average_touchInBox score')
        df_striker = calculate_score(df_striker, 'percent_successfuldPassesToFinalThird','percent_successfuldPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'shotFastbreak_per90','shotFastbreak_per90 score')
        df_striker = calculate_score(df_striker, 'bigChanceCreated_per90','bigChanceCreated_per90 score')
        df_striker = calculate_score(df_striker, 'newSuccessfulDribbles','newSuccessfulDribbles score')
        df_striker = calculate_score(df_striker, 'average_touchInBox','average_touchInBox score')
        df_striker = calculate_score(df_striker, 'average_keyPasses','average_keyPasses score')
        df_striker = calculate_score(df_striker, 'shot.xg','shot.xg score')
        df_striker = calculate_score(df_striker, 'shot.xg','shot.xg score')
        df_striker = calculate_score(df_striker, 'aerialWon','aerialWon score')


        df_striker['Linkup_play'] = df_striker[['percent_successfulPassesToFinalThird score','percent_successfulPasses score','Possession value score','average_touchInBox score','average_successfulPassesToFinalThird score','aerialWon score']].mean(axis=1)
        df_striker['Chance_creation'] = df_striker[['average_touchInBox score','Possession value total score','bigChanceCreated_per90 score','average_touchInBox score','average_successfulPassesToFinalThird score']].mean(axis=1)
        df_striker['Goalscoring_'] = df_striker[['shot.xg score','shot.xg score','shot.xg score','shot.xg score','shot.xg score']].mean(axis=1)
        df_striker['Possession_value'] = df_striker[['Possession value total score','Possession value score','Possession value score','Possession value score']].mean(axis=1)

        df_striker = calculate_score(df_striker, 'Linkup_play', 'Linkup play')
        df_striker = calculate_score(df_striker, 'Chance_creation','Chance creation')
        df_striker = calculate_score(df_striker, 'Goalscoring_','Goalscoring')        
        df_striker = calculate_score(df_striker, 'Possession_value', 'Possession value')

        
        df_striker['Total score'] = df_striker[['Linkup play','Linkup play','Linkup play','Chance creation','Goalscoring','Goalscoring','Possession value','Possession value']].mean(axis=1)
        df_striker = df_striker[['player.name','team.name','label','total_minutesOnField','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]
        df_striker = df_striker.dropna()
        df_strikertotal = df_striker[['player.name','team.name','total_minutesOnField','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]

        df_strikertotal = df_strikertotal.groupby(['player.name','team.name']).mean().reset_index()
        minutter = df_striker.groupby(['player.name', 'team.name'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_strikertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_targetman = df_striker.sort_values('Total score',ascending = False)
        df_strikertotal = df_strikertotal[['player.name','team.name','total_minutesOnField total','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]
        df_strikertotal= df_strikertotal[df_strikertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_strikertotal = df_strikertotal.sort_values('Total score',ascending = False)
        return df_targetman

    def Boxstriker():
        df_striker = df_scouting[(df_scouting['position_codes'] == 'Striker') & (df_scouting['position_codesSide'].str.contains('Centre'))]
        df_striker['total_minutesOnField'] = df_striker['total_minutesOnField'].astype(int)
        df_striker = df_striker[df_striker['total_minutesOnField'].astype(int) >= minutter_kamp]

        df_striker = calculate_score(df_striker,'average_successfulAttackingActions','Possession value total score')
        df_striker = calculate_score(df_striker,'average_successfulAttackingActions', 'Possession value score')
        df_striker = calculate_score(df_striker,'average_successfulAttackingActions', 'Possession value added score')
        df_striker = calculate_score(df_striker, 'percent_successfulPasses', 'percent_successfulPasses score')
        df_striker = calculate_score(df_striker, 'average_successfulPassesToFinalThird', 'average_successfulPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'percent_successfulPassesToFinalThird', 'percent_successfulPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'average_progressivePasses', 'average_progressivePasses score')
        df_striker = calculate_score(df_striker, 'average_shotAssists','average_shotAssists score')
        df_striker = calculate_score(df_striker, 'average_touchInBox','average_touchInBox score')
        df_striker = calculate_score(df_striker, 'percent_successfuldPassesToFinalThird','percent_successfuldPassesToFinalThird score')
        df_striker = calculate_score(df_striker, 'shotFastbreak_per90','shotFastbreak_per90 score')
        df_striker = calculate_score(df_striker, 'bigChanceCreated_per90','bigChanceCreated_per90 score')
        df_striker = calculate_score(df_striker, 'newSuccessfulDribbles','newSuccessfulDribbles score')
        df_striker = calculate_score(df_striker, 'average_touchInBox','average_touchInBox score')
        df_striker = calculate_score(df_striker, 'average_keyPasses','average_keyPasses score')
        df_striker = calculate_score(df_striker, 'shot.xg','shot.xg score')
        df_striker = calculate_score(df_striker, 'shot.xg','shot.xg score')


        df_striker['Linkup_play'] = df_striker[['percent_successfulPassesToFinalThird score','percent_successfulPasses score','Possession value score','average_touchInBox score','average_successfulPassesToFinalThird score']].mean(axis=1)
        df_striker['Chance_creation'] = df_striker[['average_touchInBox score','Possession value total score','bigChanceCreated_per90 score','average_touchInBox score','average_successfulPassesToFinalThird score']].mean(axis=1)
        df_striker['Goalscoring_'] = df_striker[['shot.xg score','shot.xg score','shot.xg score','shot.xg score','shot.xg score']].mean(axis=1)
        df_striker['Possession_value'] = df_striker[['Possession value total score','Possession value score','Possession value score','Possession value score']].mean(axis=1)

        df_striker = calculate_score(df_striker, 'Linkup_play', 'Linkup play')
        df_striker = calculate_score(df_striker, 'Chance_creation','Chance creation')
        df_striker = calculate_score(df_striker, 'Goalscoring_','Goalscoring')        
        df_striker = calculate_score(df_striker, 'Possession_value', 'Possession value')

        
        df_striker['Total score'] = df_striker[['Linkup play','Chance creation','Goalscoring','Goalscoring','Goalscoring','Goalscoring','Possession value','Possession value','Possession value']].mean(axis=1)
        df_striker = df_striker[['player.name','team.name','label','total_minutesOnField','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]
        df_striker = df_striker.dropna()
        df_strikertotal = df_striker[['player.name','team.name','total_minutesOnField','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]

        df_strikertotal = df_strikertotal.groupby(['player.name','team.name']).mean().reset_index()
        minutter = df_striker.groupby(['player.name', 'team.name'])['total_minutesOnField'].sum().astype(float).reset_index()
        df_strikertotal['total_minutesOnField total'] = minutter['total_minutesOnField']
        df_boksstriker = df_striker.sort_values('Total score',ascending = False)
        df_strikertotal = df_strikertotal[['player.name','team.name','total_minutesOnField total','Linkup play','Chance creation','Goalscoring','Possession value','Total score']]
        df_strikertotal= df_strikertotal[df_strikertotal['total_minutesOnField total'].astype(int) >= minutter_total]
        df_strikertotal = df_strikertotal.sort_values('Total score',ascending = False)
        return df_boksstriker
    return {
        'Central defender': balanced_central_defender(),
        'Fullbacks': fullbacks(),
        'Number 6' : number6(),
        'Number 8': number8(),
        'Number 10': number10(),
        'Winger': winger(),
        'Classic striker': Classic_striker(),
    }

position_dataframes = Process_data_spillere(events,df_xg,df_matchstats)

#defending_central_defender_df = position_dataframes['defending_central_defender']
#ball_playing_central_defender_df = position_dataframes['ball_playing_central_defender']
balanced_central_defender_df = position_dataframes['Central defender']
fullbacks_df = position_dataframes['Fullbacks']
number6_df = position_dataframes['Number 6']
#number6_double_6_forward_df = position_dataframes['number6_double_6_forward']
#number6_destroyer_df = position_dataframes['Number 6 (destroyer)']
number8_df = position_dataframes['Number 8']
number10_df = position_dataframes['Number 10']
winger_df = position_dataframes['Winger']
classic_striker_df = position_dataframes['Classic striker']
#targetman_df = position_dataframes['Targetman']
#box_striker_df = position_dataframes['Boxstriker']
print(balanced_central_defender_df)
balanced_central_defender_df.to_csv('balanced_central_defender.')

C:\Users\SéamusPeareBartholdy\AppData\Local\Temp\ipykernel_9780\3544581613.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xg['shot.xg'] = xg['shot.xg'].astype(float)
C:\Users\SéamusPeareBartholdy\AppData\Local\Temp\ipykernel_9780\3544581613.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_balanced_central_defender['total_minutesOnField'] = df_balanced_central_defender['total_minutesOnField'].astype(int)
C:\Users\SéamusPeareBartholdy\AppData\Local\Temp\ipykernel_9780\3544581613.py:115: SettingWit

          player.name         team.name   position_codes  \
8295        E. Nymann           AaB U19  ['rcb3', 'rcb']   
8350      M. Andersen       Brøndby U19  ['rcb3', 'rcb']   
8390          B. Kold       Brøndby U19  ['rcb3', 'rcb']   
8340          G. Nøhr       Brøndby U19  ['rcb3', 'rcb']   
8335    W. Bjergfeldt       Brøndby U19  ['rcb3', 'rcb']   
...               ...               ...              ...   
1958                0   Midtjylland U19   ['rb', 'rcb3']   
1963        G. Bjerge   Midtjylland U19   ['rb', 'rcb3']   
2073       K. Doumbia   Midtjylland U19   ['rb', 'rcb3']   
2078   D. Jóhannesson  Nordsjælland U19   ['rb', 'rcb3']   
1943  D. Kristjansson   Midtjylland U19   ['rb', 'rcb3']   

                                          label  total_minutesOnField  \
8295               AaB U19 - Brøndby U19, 2 - 4                    95   
8350               AaB U19 - Brøndby U19, 2 - 4                    95   
8390               AaB U19 - Brøndby U19, 2 - 4             